# **Imports**

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import MaxPooling2D, Dense, Flatten, Conv2D
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds

# Model

In [2]:
class Block(Model):
    
    def __init__(self, filters, kernel_size, num_layers, pool_size=2, strides=2):
        
        super().__init__()
        
        self.num_layers=num_layers
        self.filters=filters
        self.kernel_size=kernel_size
        
        for i in range(self.num_layers):
            vars(self)[f'conv2D_{i}']=Conv2D(self.filters, self.kernel_size, padding='same', activation='relu' )
            
        self.max_pool=MaxPooling2D(pool_size=pool_size, strides=strides)
            
            
    def call(self, inputs):
        
        conv2D_0= self.conv2D_0 # vars(Self).['Conv2D_0'] 
        x=conv2D_0(inputs)
        
        for i in range(1, self.num_layers):
            
            conv2d_i=vars(self)[f'conv2D_{i}']
            x=conv2d_i(x)
        
        max_pool = self.max_pool(x)
        
        return max_pool

In [3]:
class VGG(tf.keras.Model):

    def __init__(self, num_classes):
        
        super(VGG, self).__init__()
        self.block_a = Block(64, 3, 2) 
        self.block_b = Block(128, 3, 2)
        self.block_c = Block(256, 3, 3) 
        self.block_d = Block(512, 3, 3) 
        self.block_e = Block(512, 3, 3)
        self.flatten = Flatten()
        self.fc = Dense(256, activation = 'relu')
        self.classifier = Dense(num_classes, activation = 'softmax') 

    def call(self, inputs):
      
        x = self.block_a(inputs) 
        x = self.block_b(x)
        x = self.block_c(x) 
        x = self.block_d(x) 
        x = self.block_e(x)
        x = self.flatten(x) 
        x = self.fc(x)
        x = self.classifier(x) 
        return x

# Train

In [4]:
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')
vgg = VGG(num_classes=2)
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

def preprocess(features):

    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255., features['label']

dataset = dataset.map(preprocess).batch(32)
vgg.fit(dataset, epochs=10)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling data/cats_vs_dogs/4.0.0.incomplete2NDFTM/cats_vs_dogs-train.tfrecord*...:   0%|          | 0/23262 […

Dataset cats_vs_dogs downloaded and prepared to data/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
Epoch 1/10
727/727 [==============================] - 123s 154ms/step - loss: 0.6877 - accuracy: 0.5455
Epoch 2/10
727/727 [==============================] - 116s 159ms/step - loss: 0.6569 - accuracy: 0.6247
Epoch 3/10
727/727 [==============================] - 116s 159ms/step - loss: 0.6248 - accuracy: 0.6614
Epoch 4/10
727/727 [==============================] - 116s 159ms/step - loss: 0.6048 - accuracy: 0.6763
Epoch 5/10
727/727 [==============================] - 116s 159ms/step - loss: 0.5924 - accuracy: 0.6843
Epoch 6/10
727/727 [==============================] - 116s 159ms/step - loss: 0.5834 - accuracy: 0.6890
Epoch 7/10
727/727 [==============================] - 116s 159ms/step - loss: 0.5761 - accuracy: 0.6965
Epoch 8/10
727/727 [==============================] - 116s 159ms/step - loss: 0.5696 - accuracy: 0.7030
Epoch 9/10
727/727 [==============================] - 11